In [3]:
pip install torchinfo


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from khmercut import tokenize
import os

# === Step 1: Paths
captions_file_path = '/home/vitoupro/code/image_captioning/notebook/blip_captions_khmer_sorted.txt'  # raw label
output_label_path = '/home/vitoupro/code/image_captioning/data/processed/blip_captions_khmer_sorted_segment.txt'  # ⬅️ NEW segmented label

# === Step 2: Load raw captions
with open(captions_file_path, 'r', encoding='utf-8') as f:
    raw_captions = f.readlines()

captions = [line.strip() for line in raw_captions if line.strip()]

# === Step 3: Segment each caption using khmercut.tokenize
segmented_lines = []
for line in captions:
    parts = line.strip().split(' ', 1)
    if len(parts) != 2:
        print(f"⚠️ Skipping malformed line: {line}")
        continue

    image_name, caption = parts
    segmented_words = tokenize(caption)  # ✅ Use khmercut's tokenizer here
    segmented_caption = ' '.join(segmented_words)
    segmented_lines.append(f"{image_name} {segmented_caption}")

# === Step 4: Save segmented result
os.makedirs(os.path.dirname(output_label_path), exist_ok=True)

with open(output_label_path, 'w', encoding='utf-8') as f:
    for line in segmented_lines:
        f.write(line + '\n')

print(f"✅ Word-segmented captions saved to {output_label_path}")
print(f"✅ Total lines processed: {len(segmented_lines)}")


✅ Word-segmented captions saved to /home/vitoupro/code/image_captioning/data/processed/blip_captions_khmer_sorted_segment.txt
✅ Total lines processed: 4724


In [4]:
import os
import json

# === Step 1: Load segmented caption file
segmented_file_path = '/home/vitoupro/code/image_captioning/data/processed/blip_captions_khmer_sorted_segment.txt'

captions = []
with open(segmented_file_path, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split(' ', 1)
        if len(parts) == 2:
            _, caption = parts
            captions.append(caption)

# === Step 2: Tokenize into word list
tokenized_captions = [caption.split() for caption in captions]
all_words = [word for caption in tokenized_captions for word in caption]

# === Step 3: Create vocab with special tokens
special_tokens = ['<PAD>', '<START>', '<END>', '<UNK>']
vocab_words = special_tokens + sorted(set(all_words))

word2idx = {word: idx for idx, word in enumerate(vocab_words)}
idx2word = {idx: word for word, idx in word2idx.items()}

# === Step 4: Save vocab files
save_dir = '/home/vitoupro/code/image_captioning/data/processed/'
os.makedirs(save_dir, exist_ok=True)

with open(os.path.join(save_dir, 'word2idx_blip_caption_khmer_sorted.json'), 'w', encoding='utf-8') as f:
    json.dump(word2idx, f, ensure_ascii=False, indent=4)

with open(os.path.join(save_dir, 'idx2word_blip_caption_khmer_sorted.json'), 'w', encoding='utf-8') as f:
    json.dump(idx2word, f, ensure_ascii=False, indent=4)

print("✅ Vocabulary saved to:")
print("→ word2idx_blip_caption_khmer_sorted.json")
print("→ idx2word_blip_caption_khmer_sorted.json")
print(f"🟩 Total unique words (excluding special tokens): {len(vocab_words) - len(special_tokens)}")


✅ Vocabulary saved to:
→ word2idx_blip_caption_khmer_sorted.json
→ idx2word_blip_caption_khmer_sorted.json
🟩 Total unique words (excluding special tokens): 1718


In [5]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader 
from PIL import Image
import pandas as pd
import os
import re
import json
from sklearn.model_selection import train_test_split
import jiwer
import matplotlib.pyplot as plt
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from timm import create_model

# Function to load idx2word and convert it to word2idx
def load_vocabulary(path):
    with open(path, 'r') as file:
        idx2word = json.load(file)
    word2idx = {v: int(k) for k, v in idx2word.items()}
    return idx2word, word2idx

# Load vocabulary
idx2word_path = '/home/vitoupro/code/image_captioning/data/processed/idx2word_blip_caption_khmer_sorted.json'
idx2word, word2idx = load_vocabulary(idx2word_path)

# Encoding a list of words (word-level)
def encode_khmer_sentence(sentence, word2idx):
    words = sentence.strip().split()
    indices = []
    for word in words:
        index = word2idx.get(word)
        if index is None:
            return None, f"Word '{word}' not found in vocabulary!"
        indices.append(index)
    return indices, None

# Decoding a list of indices (word-level)
def decode_indices(indices, idx2word):
    words = []
    for index in indices:
        word = idx2word.get(str(index))
        if word is None:
            return None, f"Index '{index}' not found in idx2word!"
        words.append(word)
    return ' '.join(words), None

class Attention(nn.Module):
    def __init__(self, encoder_dim, decoder_dim, attention_dim):
        super(Attention, self).__init__()
        self.attn = nn.Linear(encoder_dim + decoder_dim, attention_dim)
        self.v = nn.Linear(attention_dim, 1)

    def forward(self, encoder_out, hidden):
        hidden = hidden.unsqueeze(1).repeat(1, encoder_out.size(1), 1)
        attn_input = torch.cat((encoder_out, hidden), dim=2)
        energy = torch.tanh(self.attn(attn_input))
        attention = self.v(energy).squeeze(2)
        alpha = torch.softmax(attention, dim=1)
        context = (encoder_out * alpha.unsqueeze(2)).sum(dim=1)
        return context, alpha


# Data-efficient Image Transformer (DeiT) Encoder
class DeiTEncoder(nn.Module):
    def __init__(self, embed_size):
        super(DeiTEncoder, self).__init__()
        import timm
        self.deit = timm.create_model('deit_tiny_patch16_224', pretrained=True)
        self.deit.head = nn.Identity()  # Remove classification head
        self.linear = nn.Linear(self.deit.num_features, embed_size)  # Project to embed_size

    def forward(self, images):
        features = self.deit.forward_features(images)  # (B, seq_len, 192)
        cls_token = features[:, 0, :]  # Take only [CLS] token: (B, 192)
        out = self.linear(cls_token)  # Project to (B, embed_size)
        return out.unsqueeze(1)  # (B, 1, embed_size)
       


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :].to(x.device)

class TransformerDecoder(nn.Module):
    def __init__(self, vocab_size, embed_size, num_heads=8, num_layers=3, ff_dim=512, dropout=0.1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.pos_encoding = PositionalEncoding(embed_size)
        decoder_layer = nn.TransformerDecoderLayer(d_model=embed_size, nhead=num_heads, dim_feedforward=ff_dim, dropout=dropout)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(embed_size, vocab_size)

    def generate_square_subsequent_mask(self, sz):
        return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

    def forward(self, features, captions):
        tgt = self.embedding(captions)
        tgt = self.pos_encoding(tgt)
        tgt = tgt.permute(1, 0, 2)
        memory = features.permute(1, 0, 2)
        tgt_mask = self.generate_square_subsequent_mask(tgt.size(0)).to(tgt.device)
        out = self.transformer_decoder(tgt, memory, tgt_mask=tgt_mask)
        out = self.fc_out(out)
        return out.permute(1, 0, 2)

class ImageCaptionDataset(torch.utils.data.Dataset):
    def __init__(self, img_labels, img_dir, vocab, transform=None, max_length=50):
        self.img_labels = img_labels
        self.img_dir = img_dir
        self.vocab = vocab
        self.transform = transform
        self.max_length = max_length

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        caption = self.img_labels.iloc[idx, 1]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        indices, error = encode_khmer_sentence(caption, self.vocab)
        if error:
            print(f"Error encoding caption: {error}")
            indices = [self.vocab['<UNK>']] * self.max_length
        tokens = [self.vocab['<START>']] + indices + [self.vocab['<END>']]
        tokens += [self.vocab['<PAD>']] * (self.max_length - len(tokens))
        return image, torch.tensor(tokens[:self.max_length])

transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

annotations_file = '/home/vitoupro/code/image_captioning/data/processed/blip_captions_khmer_sorted_segment.txt'
img_dir = '/home/vitoupro/code/image_captioning/notebook/downloaded_images'
image_names, captions = [], []

with open(annotations_file, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split(' ', 1)
        if len(parts) == 2:
            image_name, caption = parts
            image_names.append(image_name)
            captions.append(caption)

all_images = pd.DataFrame({'image': image_names, 'caption': captions})
train_images, eval_images, train_captions, eval_captions = train_test_split(
    all_images['image'].tolist(), all_images['caption'].tolist(), test_size=0.2, random_state=42
)

train_dataset = ImageCaptionDataset(pd.DataFrame({'image': train_images, 'caption': train_captions}), img_dir, word2idx, transform, 20)
eval_dataset = ImageCaptionDataset(pd.DataFrame({'image': eval_images, 'caption': eval_captions}), img_dir, word2idx, transform, 20)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=32, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
embed_size = 256
encoder = DeiTEncoder(embed_size=embed_size).to(device)
decoder = TransformerDecoder(len(word2idx), embed_size, num_heads=8, num_layers=3).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=word2idx['<PAD>'])
params = list(decoder.parameters()) + list(encoder.parameters())
optimizer = torch.optim.Adam(params, lr=0.001, weight_decay=1e-5)

smoothing = SmoothingFunction().method1
rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=False)

def evaluate_model(encoder, decoder, dataloader, device, epoch):
    encoder.eval()
    decoder.eval()
    total_wer, total_cer, total_bleu1, total_bleu2, total_bleu4, total_rougeL = 0, 0, 0, 0, 0, 0
    num_samples = 0

    with torch.no_grad():
        for images, captions in dataloader:
            images, captions = images.to(device), captions.to(device)
            features = encoder(images)
            outputs = decoder(features, captions[:, :-1])
            predicted_captions = outputs.argmax(-1)

            for i in range(len(captions)):
                gt_caption = decode_indices(captions[i].tolist(), idx2word)[0]
                pred_caption = decode_indices(predicted_captions[i].tolist(), idx2word)[0]

                ref_match = re.search(r"<START>(.*?)<END>", gt_caption)
                pred_match = re.search(r"^(.*?)<END>", pred_caption)
                reference = ref_match.group(1).strip() if ref_match else ""
                prediction = pred_match.group(1).strip() if pred_match else ""
                if not reference or not prediction:
                    continue

                ref_tokens = reference.split()
                pred_tokens = prediction.split()

                total_wer += jiwer.wer(reference, prediction)
                total_cer += jiwer.cer(reference, prediction)
                total_bleu1 += sentence_bleu([ref_tokens], pred_tokens, weights=(1, 0, 0, 0), smoothing_function=smoothing)
                total_bleu2 += sentence_bleu([ref_tokens], pred_tokens, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing)
                total_bleu4 += sentence_bleu([ref_tokens], pred_tokens, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothing)
                total_rougeL += rouge.score(reference, prediction)['rougeL'].fmeasure
                num_samples += 1

    avg_wer = total_wer / num_samples if num_samples > 0 else 0
    avg_cer = total_cer / num_samples if num_samples > 0 else 0
    avg_bleu1 = total_bleu1 / num_samples if num_samples > 0 else 0
    avg_bleu2 = total_bleu2 / num_samples if num_samples > 0 else 0
    avg_bleu4 = total_bleu4 / num_samples if num_samples > 0 else 0
    avg_rougeL = total_rougeL / num_samples if num_samples > 0 else 0

    print(f"WER: {avg_wer:.2f}, CER: {avg_cer:.2f}, BLEU-1: {avg_bleu1:.2f}, BLEU-2: {avg_bleu2:.2f}, BLEU-4: {avg_bleu4:.2f}, ROUGE-L: {avg_rougeL:.2f}")
    return avg_wer, avg_cer, avg_bleu1, avg_bleu2, avg_bleu4, avg_rougeL


/home/vitoupro/code/image_captioning/image_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
num_epochs = 15
best_wer = float('inf')
teacher_forcing_ratio = 0.9  # Start with 90% teacher forcing

train_losses, wer_scores, cer_scores = [], [], []
bleu1_scores, bleu2_scores, bleu4_scores, rougeL_scores = [], [], [], []

for epoch in range(num_epochs):
    encoder.train()
    decoder.train()
    total_loss = 0

    for images, captions in train_loader:
        images, captions = images.to(device), captions.to(device)

        features = encoder(images)
        
        input_tokens = captions[:, :-1]
        targets = captions[:, 1:]

        if torch.rand(1).item() > teacher_forcing_ratio:
            # Scheduled sampling: use own prediction as input
            outputs = []
            batch_size = images.size(0)
            inputs = torch.full((batch_size, 1), word2idx['<START>'], dtype=torch.long, device=device)

            for _ in range(input_tokens.size(1)):
                output = decoder(features, inputs)
                last_output = output[:, -1, :]  # last timestep
                predicted = last_output.argmax(-1).unsqueeze(1)
                inputs = torch.cat((inputs, predicted), dim=1)
                outputs.append(last_output)

            outputs = torch.stack(outputs, dim=1).squeeze(2)  # (B, T, vocab)
        else:
            # Teacher forcing: normal
            outputs = decoder(features, input_tokens)

        batch_size, seq_len, vocab_size = outputs.size()
        loss = criterion(
            outputs.reshape(batch_size * seq_len, vocab_size),
            targets.reshape(-1)
        )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)

    print(f"Epoch {epoch+1}: Train Loss: {avg_loss:.4f}")

    avg_wer, avg_cer, avg_bleu1, avg_bleu2, avg_bleu4, avg_rougeL = evaluate_model(encoder, decoder, eval_loader, device, epoch)

    wer_scores.append(avg_wer)
    cer_scores.append(avg_cer)

    # Reduce teacher forcing slowly
    teacher_forcing_ratio = max(0.3, teacher_forcing_ratio * 0.9)


Epoch 1: Train Loss: 3.9495
WER: 0.72, CER: 0.64, BLEU-1: 0.29, BLEU-2: 0.18, BLEU-4: 0.08, ROUGE-L: 0.00
Epoch 2: Train Loss: 3.3265
WER: 0.68, CER: 0.60, BLEU-1: 0.33, BLEU-2: 0.20, BLEU-4: 0.11, ROUGE-L: 0.00
Epoch 3: Train Loss: 3.2123
WER: 0.68, CER: 0.60, BLEU-1: 0.34, BLEU-2: 0.22, BLEU-4: 0.10, ROUGE-L: 0.01
Epoch 4: Train Loss: 3.3382
WER: 0.66, CER: 0.59, BLEU-1: 0.35, BLEU-2: 0.23, BLEU-4: 0.11, ROUGE-L: 0.01
Epoch 5: Train Loss: 3.3967
WER: 0.65, CER: 0.58, BLEU-1: 0.37, BLEU-2: 0.24, BLEU-4: 0.12, ROUGE-L: 0.01
Epoch 6: Train Loss: 3.6058
WER: 0.68, CER: 0.60, BLEU-1: 0.34, BLEU-2: 0.21, BLEU-4: 0.11, ROUGE-L: 0.01
Epoch 7: Train Loss: 3.5711
WER: 0.64, CER: 0.57, BLEU-1: 0.37, BLEU-2: 0.25, BLEU-4: 0.12, ROUGE-L: 0.01
Epoch 8: Train Loss: 3.4291
WER: 0.65, CER: 0.58, BLEU-1: 0.36, BLEU-2: 0.24, BLEU-4: 0.12, ROUGE-L: 0.01
Epoch 9: Train Loss: 3.6365
WER: 0.65, CER: 0.59, BLEU-1: 0.38, BLEU-2: 0.25, BLEU-4: 0.12, ROUGE-L: 0.01
Epoch 10: Train Loss: 3.7296
WER: 0.65, CER: 0

In [ ]:
@torch.no_grad()
def predict_caption(image_path, encoder, decoder, transform, device, idx2word, word2idx, max_length=20):
    encoder.eval()
    decoder.eval()

    # Load and transform image
    image = Image.open(image_path).convert('RGB')
    if transform:
        image = transform(image)
    image = image.unsqueeze(0).to(device)  # shape (1, 3, H, W)

    # Encode image
    encoder_out = encoder(image)  # (1, 1, embed_size)

    # Start generating
    generated_indices = [word2idx['<START>']]
    
    for _ in range(max_length):
        input_tensor = torch.tensor([generated_indices], dtype=torch.long).to(device)  # (1, T)
        
        # Predict next token
        output = decoder(encoder_out, input_tensor)  # (1, T, vocab_size)
        next_token_logits = output[:, -1, :]          # (1, vocab_size)
        predicted_index = next_token_logits.argmax(dim=-1).item()

        if predicted_index == word2idx['<END>']:
            break
        
        generated_indices.append(predicted_index)

    # Decode indices to words
    predicted_tokens = [idx2word[str(idx)] for idx in generated_indices[1:]]  # Skip <START>

    predicted_caption = ' '.join(predicted_tokens)  # 🔥 join words with space
    return predicted_caption

print("\n=== Testing a prediction after training ===\n")

test_image_path = '/home/vitoupro/code/image_captioning/data/image.png'  # your image

predicted_caption = predict_caption(
    image_path=test_image_path,
    encoder=encoder,
    decoder=decoder,
    transform=transform,
    device=device,
    idx2word=idx2word,
    word2idx=word2idx,
    max_length=20
)

print("Predicted Caption:", predicted_caption)


=== Testing a prediction after training ===

Predicted Caption: ផ្លូវ ម្នាក់ ដែល មាន នៅលើ នៅលើ


In [5]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader 
from PIL import Image
import pandas as pd
import os
import re
import json
from sklearn.model_selection import train_test_split
import jiwer
import matplotlib.pyplot as plt
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from timm import create_model

# Function to load idx2word and convert it to word2idx
def load_vocabulary(path):
    with open(path, 'r') as file:
        idx2word = json.load(file)
    word2idx = {v: int(k) for k, v in idx2word.items()}
    return idx2word, word2idx

# Load vocabulary
idx2word_path = '/home/vitoupro/code/image_captioning/data/processed/idx2word_blip_caption_khmer_sorted.json'
idx2word, word2idx = load_vocabulary(idx2word_path)

# Encoding a list of words (word-level)
def encode_khmer_sentence(sentence, word2idx):
    words = sentence.strip().split()
    indices = []
    for word in words:
        index = word2idx.get(word)
        if index is None:
            return None, f"Word '{word}' not found in vocabulary!"
        indices.append(index)
    return indices, None

# Decoding a list of indices (word-level)
def decode_indices(indices, idx2word):
    words = []
    for index in indices:
        word = idx2word.get(str(index))
        if word is None:
            return None, f"Index '{index}' not found in idx2word!"
        words.append(word)
    return ' '.join(words), None

class Attention(nn.Module):
    def __init__(self, encoder_dim, decoder_dim, attention_dim):
        super(Attention, self).__init__()
        self.attn = nn.Linear(encoder_dim + decoder_dim, attention_dim)
        self.v = nn.Linear(attention_dim, 1)

    def forward(self, encoder_out, hidden):
        hidden = hidden.unsqueeze(1).repeat(1, encoder_out.size(1), 1)
        attn_input = torch.cat((encoder_out, hidden), dim=2)
        energy = torch.tanh(self.attn(attn_input))
        attention = self.v(energy).squeeze(2)
        alpha = torch.softmax(attention, dim=1)
        context = (encoder_out * alpha.unsqueeze(2)).sum(dim=1)
        return context, alpha


# Data-efficient Image Transformer (DeiT) Encoder
class DeiTEncoder(nn.Module):
    def __init__(self, embed_size):
        super(DeiTEncoder, self).__init__()
        import timm
        self.deit = timm.create_model('deit_small_patch16_224', pretrained=True)
        self.deit.head = nn.Identity()  # Remove classification head
        self.linear = nn.Linear(self.deit.num_features, embed_size)  # Project to embed_size

    def forward(self, images):
        features = self.deit.forward_features(images)  # (B, seq_len, 192)
        cls_token = features[:, 0, :]  # Take only [CLS] token: (B, 192)
        out = self.linear(cls_token)  # Project to (B, embed_size)
        return out.unsqueeze(1)  # (B, 1, embed_size)
       


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :].to(x.device)

class TransformerDecoder(nn.Module):
    def __init__(self, vocab_size, embed_size, num_heads=8, num_layers=3, ff_dim=512, dropout=0.1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.pos_encoding = PositionalEncoding(embed_size)
        decoder_layer = nn.TransformerDecoderLayer(d_model=embed_size, nhead=num_heads, dim_feedforward=ff_dim, dropout=dropout)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(embed_size, vocab_size)

    def generate_square_subsequent_mask(self, sz):
        return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

    def forward(self, features, captions):
        tgt = self.embedding(captions)
        tgt = self.pos_encoding(tgt)
        tgt = tgt.permute(1, 0, 2)
        memory = features.permute(1, 0, 2)
        tgt_mask = self.generate_square_subsequent_mask(tgt.size(0)).to(tgt.device)
        out = self.transformer_decoder(tgt, memory, tgt_mask=tgt_mask)
        out = self.fc_out(out)
        return out.permute(1, 0, 2)

class ImageCaptionDataset(torch.utils.data.Dataset):
    def __init__(self, img_labels, img_dir, vocab, transform=None, max_length=50):
        self.img_labels = img_labels
        self.img_dir = img_dir
        self.vocab = vocab
        self.transform = transform
        self.max_length = max_length

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        caption = self.img_labels.iloc[idx, 1]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        indices, error = encode_khmer_sentence(caption, self.vocab)
        if error:
            print(f"Error encoding caption: {error}")
            indices = [self.vocab['<UNK>']] * self.max_length
        tokens = [self.vocab['<START>']] + indices + [self.vocab['<END>']]
        tokens += [self.vocab['<PAD>']] * (self.max_length - len(tokens))
        return image, torch.tensor(tokens[:self.max_length])

transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

annotations_file = '/home/vitoupro/code/image_captioning/data/processed/blip_captions_khmer_sorted_segment.txt'
img_dir = '/home/vitoupro/code/image_captioning/notebook/downloaded_images'
image_names, captions = [], []

with open(annotations_file, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split(' ', 1)
        if len(parts) == 2:
            image_name, caption = parts
            image_names.append(image_name)
            captions.append(caption)

all_images = pd.DataFrame({'image': image_names, 'caption': captions})
train_images, eval_images, train_captions, eval_captions = train_test_split(
    all_images['image'].tolist(), all_images['caption'].tolist(), test_size=0.2, random_state=42
)

train_dataset = ImageCaptionDataset(pd.DataFrame({'image': train_images, 'caption': train_captions}), img_dir, word2idx, transform, 20)
eval_dataset = ImageCaptionDataset(pd.DataFrame({'image': eval_images, 'caption': eval_captions}), img_dir, word2idx, transform, 20)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=32, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
embed_size = 256
encoder = DeiTEncoder(embed_size=embed_size).to(device)
decoder = TransformerDecoder(len(word2idx), embed_size, num_heads=8, num_layers=3).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=word2idx['<PAD>'])
params = list(decoder.parameters()) + list(encoder.parameters())
optimizer = torch.optim.Adam(params, lr=0.001, weight_decay=1e-5)

smoothing = SmoothingFunction().method1
rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=False)

def evaluate_model(encoder, decoder, dataloader, device, epoch):
    encoder.eval()
    decoder.eval()
    total_wer, total_cer, total_bleu1, total_bleu2, total_bleu4, total_rougeL = 0, 0, 0, 0, 0, 0
    num_samples = 0

    with torch.no_grad():
        for images, captions in dataloader:
            images, captions = images.to(device), captions.to(device)
            features = encoder(images)
            outputs = decoder(features, captions[:, :-1])
            predicted_captions = outputs.argmax(-1)

            for i in range(len(captions)):
                gt_caption = decode_indices(captions[i].tolist(), idx2word)[0]
                pred_caption = decode_indices(predicted_captions[i].tolist(), idx2word)[0]

                ref_match = re.search(r"<START>(.*?)<END>", gt_caption)
                pred_match = re.search(r"^(.*?)<END>", pred_caption)
                reference = ref_match.group(1).strip() if ref_match else ""
                prediction = pred_match.group(1).strip() if pred_match else ""
                if not reference or not prediction:
                    continue

                ref_tokens = reference.split()
                pred_tokens = prediction.split()

                total_wer += jiwer.wer(reference, prediction)
                total_cer += jiwer.cer(reference, prediction)
                total_bleu1 += sentence_bleu([ref_tokens], pred_tokens, weights=(1, 0, 0, 0), smoothing_function=smoothing)
                total_bleu2 += sentence_bleu([ref_tokens], pred_tokens, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing)
                total_bleu4 += sentence_bleu([ref_tokens], pred_tokens, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothing)
                total_rougeL += rouge.score(reference, prediction)['rougeL'].fmeasure
                num_samples += 1

    avg_wer = total_wer / num_samples if num_samples > 0 else 0
    avg_cer = total_cer / num_samples if num_samples > 0 else 0
    avg_bleu1 = total_bleu1 / num_samples if num_samples > 0 else 0
    avg_bleu2 = total_bleu2 / num_samples if num_samples > 0 else 0
    avg_bleu4 = total_bleu4 / num_samples if num_samples > 0 else 0
    avg_rougeL = total_rougeL / num_samples if num_samples > 0 else 0

    print(f"WER: {avg_wer:.2f}, CER: {avg_cer:.2f}, BLEU-1: {avg_bleu1:.2f}, BLEU-2: {avg_bleu2:.2f}, BLEU-4: {avg_bleu4:.2f}, ROUGE-L: {avg_rougeL:.2f}")
    return avg_wer, avg_cer, avg_bleu1, avg_bleu2, avg_bleu4, avg_rougeL


In [14]:
print("✅ Dncoder Type:", type(encoder))
print("✅ Number of Parameters in Encoder:", sum(p.numel() for p in encoder.parameters()))
print("✅ Number of Trainable Parameters:", sum(p.numel() for p in encoder.parameters() if p.requires_grad))


✅ Dncoder Type: <class '__main__.DeiTEncoder'>
✅ Number of Parameters in Encoder: 21764224
✅ Number of Trainable Parameters: 21764224


In [12]:
print("✅ Decoder Type:", type(decoder))
print("✅ Number of Parameters in Decoder:", sum(p.numel() for p in decoder.parameters()))
print("✅ Number of Trainable Parameters:", sum(p.numel() for p in decoder.parameters() if p.requires_grad))


✅ Decoder Type: <class '__main__.TransformerDecoder'>
✅ Number of Parameters in Decoder: 3255738
✅ Number of Trainable Parameters: 3255738


In [15]:
num_epochs = 15
best_wer = float('inf')
teacher_forcing_ratio = 0.9  # Start with 90% teacher forcing

train_losses, wer_scores, cer_scores = [], [], []
bleu1_scores, bleu2_scores, bleu4_scores, rougeL_scores = [], [], [], []

for epoch in range(num_epochs):
    encoder.train()
    decoder.train()
    total_loss = 0

    for images, captions in train_loader:
        images, captions = images.to(device), captions.to(device)

        features = encoder(images)
        
        input_tokens = captions[:, :-1]
        targets = captions[:, 1:]

        if torch.rand(1).item() > teacher_forcing_ratio:
            # Scheduled sampling: use own prediction as input
            outputs = []
            batch_size = images.size(0)
            inputs = torch.full((batch_size, 1), word2idx['<START>'], dtype=torch.long, device=device)

            for _ in range(input_tokens.size(1)):
                output = decoder(features, inputs)
                last_output = output[:, -1, :]  # last timestep
                predicted = last_output.argmax(-1).unsqueeze(1)
                inputs = torch.cat((inputs, predicted), dim=1)
                outputs.append(last_output)

            outputs = torch.stack(outputs, dim=1).squeeze(2)  # (B, T, vocab)
        else:
            # Teacher forcing: normal
            outputs = decoder(features, input_tokens)

        batch_size, seq_len, vocab_size = outputs.size()
        loss = criterion(
            outputs.reshape(batch_size * seq_len, vocab_size),
            targets.reshape(-1)
        )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)

    print(f"Epoch {epoch+1}: Train Loss: {avg_loss:.4f}")

    avg_wer, avg_cer, avg_bleu1, avg_bleu2, avg_bleu4, avg_rougeL = evaluate_model(encoder, decoder, eval_loader, device, epoch)

    wer_scores.append(avg_wer)
    cer_scores.append(avg_cer)

    # Reduce teacher forcing slowly
    teacher_forcing_ratio = max(0.3, teacher_forcing_ratio * 0.9)


Epoch 1: Train Loss: 4.0593
WER: 0.73, CER: 0.65, BLEU-1: 0.28, BLEU-2: 0.17, BLEU-4: 0.09, ROUGE-L: 0.00
Epoch 2: Train Loss: 3.3484
WER: 0.70, CER: 0.62, BLEU-1: 0.31, BLEU-2: 0.19, BLEU-4: 0.09, ROUGE-L: 0.00
Epoch 3: Train Loss: 3.3634
WER: 0.68, CER: 0.61, BLEU-1: 0.31, BLEU-2: 0.20, BLEU-4: 0.10, ROUGE-L: 0.01
Epoch 4: Train Loss: 3.1843
WER: 0.68, CER: 0.61, BLEU-1: 0.33, BLEU-2: 0.21, BLEU-4: 0.10, ROUGE-L: 0.01
Epoch 5: Train Loss: 3.2915
WER: 0.65, CER: 0.58, BLEU-1: 0.37, BLEU-2: 0.24, BLEU-4: 0.12, ROUGE-L: 0.01
Epoch 6: Train Loss: 3.4639
WER: 0.74, CER: 0.65, BLEU-1: 0.27, BLEU-2: 0.16, BLEU-4: 0.08, ROUGE-L: 0.01
Epoch 7: Train Loss: 3.5978
WER: 0.66, CER: 0.59, BLEU-1: 0.37, BLEU-2: 0.24, BLEU-4: 0.12, ROUGE-L: 0.01
Epoch 8: Train Loss: 3.5317
WER: 0.67, CER: 0.61, BLEU-1: 0.35, BLEU-2: 0.22, BLEU-4: 0.11, ROUGE-L: 0.01
Epoch 9: Train Loss: 3.6541
WER: 0.65, CER: 0.58, BLEU-1: 0.37, BLEU-2: 0.25, BLEU-4: 0.12, ROUGE-L: 0.01
Epoch 10: Train Loss: 3.7834
WER: 0.67, CER: 0

In [16]:
if avg_wer < best_wer:
    best_wer = avg_wer
    torch.save(encoder.state_dict(), "encoderDeiT.pth")
    torch.save(decoder.state_dict(), "decodertransf.pth")
    print("✅ Saved best model!")

✅ Saved best model!


In [17]:
# Save the model state dictionary
torch.save({
    'encoder_state_dict': encoder.state_dict(),
    'decoder_state_dict': decoder.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, 'captioning_model_transf.pth')


In [22]:
@torch.no_grad()
def predict_caption(image_path, encoder, decoder, transform, device, idx2word, word2idx, max_length=20):
    encoder.eval()
    decoder.eval()

    # Load and transform image
    image = Image.open(image_path).convert('RGB')
    if transform:
        image = transform(image)
    image = image.unsqueeze(0).to(device)  # shape (1, 3, H, W)

    # Encode image
    encoder_out = encoder(image)  # (1, 1, embed_size)

    # Start generating
    generated_indices = [word2idx['<START>']]
    
    for _ in range(max_length):
        input_tensor = torch.tensor([generated_indices], dtype=torch.long).to(device)  # (1, T)
        
        # Predict next token
        output = decoder(encoder_out, input_tensor)  # (1, T, vocab_size)
        next_token_logits = output[:, -1, :]          # (1, vocab_size)
        predicted_index = next_token_logits.argmax(dim=-1).item()

        if predicted_index == word2idx['<END>']:
            break
        
        generated_indices.append(predicted_index)

    # Decode indices to words
    predicted_tokens = [idx2word[str(idx)] for idx in generated_indices[1:]]  # Skip <START>

    predicted_caption = ' '.join(predicted_tokens)  # 🔥 join words with space
    return predicted_caption

print("\n=== Testing a prediction after training ===\n")

test_image_path = '/home/vitoupro/code/image_captioning/data/10.png'  # your image

predicted_caption = predict_caption(
    image_path=test_image_path,
    encoder=encoder,
    decoder=decoder,
    transform=transform,
    device=device,
    idx2word=idx2word,
    word2idx=word2idx,
    max_length=20
)

print("Predicted Caption:", predicted_caption)


=== Testing a prediction after training ===

Predicted Caption: ស្ត្រី ម្នាក់ មាន មាន ពណ៌


In [24]:
import torch
from PIL import Image
from torchvision import transforms

# === Load trained model ===
encoder.load_state_dict(torch.load('/home/vitoupro/code/image_captioning/notebook/encoderDeiT.pth'))  # Replace with actual path
decoder.load_state_dict(torch.load('/home/vitoupro/code/image_captioning/notebook/decodertransf.pth'))  # Replace with actual path
encoder.eval()
decoder.eval()
encoder.to(device)
decoder.to(device)

# === Preprocessing ===
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# === Caption Generation ===
@torch.no_grad()
def predict_caption(image_path, encoder, decoder, transform, device, idx2word, word2idx, max_length=20):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # (1, 3, 224, 224)

    features = encoder(image)  # (1, 1, embed_size)
    generated_ids = [word2idx["<START>"]]

    for _ in range(max_length):
        input_seq = torch.tensor([generated_ids], dtype=torch.long).to(device)
        outputs = decoder(features, input_seq)  # (1, T, vocab_size)
        next_token_logits = outputs[:, -1, :]
        predicted_id = next_token_logits.argmax(-1).item()

        if predicted_id == word2idx["<END>"]:
            break
        generated_ids.append(predicted_id)

    tokens = [idx2word[str(i)] for i in generated_ids[1:]]  # skip <START>
    return " ".join(tokens)

# === Run Inference ===
test_image = "/home/vitoupro/code/image_captioning/data/10.png"  # Replace with your test image
caption = predict_caption(test_image, encoder, decoder, transform, device, idx2word, word2idx)
print("🖼️ Predicted Caption:", caption)


🖼️ Predicted Caption: ស្ត្រី ម្នាក់ ដែល មាន ពណ៌
